In [33]:
import netCDF4 as nc
import numpy as np
import arrow


In [34]:
## are the non dic cells of the PREIND BCs equivalent to the liveocean ones and are the 2 LLO equivalent to each other?

LO = \
nc.Dataset('/results/forcing/LiveOcean/boundary_conditions/LiveOcean_v201905_y2015m04d10.nc')
LO2 = \
nc.Dataset('/results/forcing/LiveOcean/boundary_conditions/LiveOcean_v201712_y2015m04d10.nc')

PI = nc.Dataset('./PREIND_BCCOND/PREIND_BC_v201905_y2015m04d10.nc')
w = LO['TA'][:]-LO2['TA'][:]
w2 = LO['TA'][:]-PI['TA'][:]
print(np.min(w2))
# print(LO2)

0.0


In [35]:
start ='2015-01-01'
end ='2015-12-31'

start_run = arrow.get(start)
end_run = arrow.get(end)

arrow_array = []

for r in arrow.Arrow.span_range('day', start_run, end_run):
    arrow_array.append(r)

In [36]:
newPI_DIC = np.zeros([365,40,950])
newPI_sigma0 = np.zeros([365,40,950])
for i in range(0,365):

    tdate = arrow_array[i][0]
    yy = tdate.format('YYYY')
    mm = tdate.format('MM')
    dd = tdate.format('DD')
    ymd = f'y{yy}m{mm}d{dd}'

#     if i%60 == 0:
        #print(ymd)
    newPI = nc.Dataset(f'./preind_DIC/LO_AOUmethod_stoicCO_diseq_allup_{ymd}_preind_DIC.nc')
   # print(newPI)
    newPI_DIC[i,:,:] = newPI['preindustrial_DIC']['preind_dic'][:,:] 
    newPI_sigma0[i,:,:] = newPI['preindustrial_DIC']['sigma0'][:,:] 
    newPI.close()

In [37]:
#retreive present day DIC and salinity
presday_DIC = np.zeros([365,40,950])
presday_TA = np.zeros([365,40,950])
presday_sal = np.zeros([365,40,950])

for i in range(0,365):

    tdate = arrow_array[i][0]
    #ymd = tdate.format('YYYY')
    yy = tdate.format('YYYY')
    mm = tdate.format('MM')
    dd = tdate.format('DD')
    ymd = f'y{yy}m{mm}d{dd}.nc'

#     if i%60 == 0:
#         #print(ymd)
    presday = nc.Dataset(f'/results/forcing/LiveOcean/boundary_conditions/LiveOcean_v201905_{ymd}')
    presday_DIC[i,:,:] = presday['DIC'][0,:,0,:]
    presday_TA[i,:,:] = presday['TA'][0,:,0,:]
    presday_sal[i,:,:] = presday['vosaline'][0,:,0,:]
    presday.close()



In [38]:
def toy_interpolator(sigma0_prof,PI_dic_prof, presday_dic_prof, surf_dic):
    
#tp testprof
    presday_DIC_tp = presday_dic_prof
    newPI_DIC_tp = PI_dic_prof
    newPI_sigma0_tp = sigma0_prof
    
#     print(newPI_sigma0_tp)

    min_over25 = (np.min(newPI_sigma0_tp[newPI_sigma0_tp>25]))
    loc_min_over25 = (np.where(newPI_sigma0_tp == min_over25))
    intrusion_at_min = presday_DIC_tp[loc_min_over25] - \
        newPI_DIC_tp[loc_min_over25]

    # print(intrusion_at_min)

    #print(depths[loc_min_over25])
    surf_int = 60
    diff = surf_int - intrusion_at_min
    depth_way = depths/depths[loc_min_over25]

    #print(depth_way)

    surf_intr = np.zeros_like(newPI_sigma0_tp)
    surf_intr = surf_int - (depth_way) * (diff)
    #print(surf_intr)
    #print(surf_intr[22])

    newPI_DIC_surfadj_tp = np.copy(newPI_DIC_tp)
    #print(loc_min_over25[0][0])
    newPI_DIC_surfadj_tp[0:loc_min_over25[0][0]] = \
    presday_DIC_tp[0:loc_min_over25[0][0]]-surf_intr[0:loc_min_over25[0][0]]
    
    return newPI_DIC_surfadj_tp


In [39]:
w = nc.Dataset('/data/tjarniko/MEOPAR/grid/mesh_mask201702.nc')
depths = w['gdept_1d'][0,:]

#newPI_sigma0
newPI_DIC_surfadj = np.copy(newPI_DIC)
print(np.shape(newPI_DIC_surfadj))

for d in range(0,365):
    if d%60 == 0:
        print(d)
    for w in range(0,950):
        
        surfadj =\
        toy_interpolator(newPI_sigma0[d,:,w],newPI_DIC[d,:,w], presday_DIC[d,:,w], 60)
#         print(np.shape(surfadj))
#         print(surfadj)
        newPI_DIC_surfadj[d,:,w] = surfadj
        
        

(365, 40, 950)
0
60
120
180
240
300
360


In [57]:
for i in range(0,365):

    tdate = arrow_array[i][0]
    yy = tdate.format('YYYY')
    mm = tdate.format('MM')
    dd = tdate.format('DD')
    ymd = f'y{yy}m{mm}d{dd}'
    
    #input file
    test_LO = nc.Dataset(f'/results/forcing/LiveOcean/boundary_conditions/LiveOcean_v201905_{ymd}.nc')
    #print(test_LO)
    #input file for preindustrial DIC
    dsin = test_LO 
    #output filename
    ncname = f'./PREIND_BCCOND/PI_AOU_SIGADJ_v201905_{ymd}.nc'

    dsout = nc.Dataset(ncname, "w", format="NETCDF4")
    #Copy dimensions
    for dname, the_dim in dsin.dimensions.items():
        #print(dname)
        #print(len(the_dim))
        dsout.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)
        
    # Copy variables
    for v_name, varin in dsin.variables.items():
        #print(v_name)
        #if (v_name != 'DIC'):
        outVar = dsout.createVariable(v_name, varin.datatype, varin.dimensions)
        # Copy variable attributes
        outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs()})

        outVar[:] = varin[:]
    
    # copy new DIC variables
    piDIC = newPI_DIC_surfadj[i,:,:]
    t_DIC = np.zeros([1, 40, 1, 950])
    t_DIC[0,:,0,:] = piDIC[:,:]    
    dsout['DIC'][:] = t_DIC

    dsout.close()